In [11]:
from __future__ import print_function
import sys
sys.path.append(".")
from models.naive_dcgan import Generator, Discriminator, init_weights
from utils.image_utils import get_tensor_image, save_tensor_images
import os
import argparse
import torch
import torch.utils as tutils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn as nn
import torchvision.utils as tvutils
import numpy as np
from cv2 import imread, resize

import random
use_cuda = torch.cuda.is_available()

    # Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

torch.cuda.is_available()

CUDA is supported


True

In [12]:
device = torch.device("cuda:0")
num_gpu = 1
num_gf = 64
num_df = 64
num_iters = 1000
lr = 0.01
num_iters = 1000
z_dim = 100
lamd = 0.1
image_size = 64
batch_size = 64
# number of image channels
num_channels = 3
output_dir = "./results"
netG_path = "./generator_epoch_000.pth"
netD_path = "./discriminator_epoch_000.pth"
aligned_images = "./dataset" #需要进行impainting的测试图像集合
use_cuda = True
########need to add path#########
aligned_images = []
for r,d,f in os.walk("./dataset/img_align_celeba/"):
    aligned_images = f
    
print(len(aligned_images))
assert aligned_images



#################

# set cudnn benchmark=True ,for the best of the backend to find best hardware algorithms
cudnn.benchmark = True

generator = Generator(z_dim, num_gf, num_channels, use_cuda, num_gpu)
discriminator = Discriminator(num_channels, num_df, use_cuda, num_gpu)

# move generator and disciminator to cuda device if cuda is activated
generator.to(device)
discriminator.to(device)

# init weights of  g and d

generator.apply(init_weights)
discriminator.apply(init_weights)

# load net state if exists
print("load trained state dict from local files...")
generator.load_state_dict(torch.load(netG_path))
discriminator.load_state_dict(torch.load(netD_path))
print("generator and discriminator state dict loaded, done.")

print("Generator Info:")
print(generator)
print("Discriminator Info:")
print(discriminator)

image_shape = [num_channels, image_size, image_size]

criteria = nn.BCELoss()


200000
load trained state dict from local files...
generator and discriminator state dict loaded, done.
Generator Info:
Generator(
  (net): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTransp

In [13]:
aligned_images = aligned_images[0:10]
for i in range(len(aligned_images)):
    aligned_images[i] = "./dataset/img_align_celeba/" + aligned_images[i]
    

In [8]:
import cv2

In [ ]:
cv2.resize()

In [40]:
def impainting():
    # load mask
    mask_img = imread("./qd_imd/train/14508_train.png")
    mask_img_ds = resize(mask_img, (64,64))
    # 创建输出文件夹
    output_dirs = output_dir
    source_imagedir = os.path.join(output_dirs, "source_images")
    masked_imagedir = os.path.join(output_dirs, "masked_images")
    impainted_imagedir = os.path.join(output_dirs, "impainted_images")
    os.makedirs(source_imagedir, exist_ok=True)
    os.makedirs(masked_imagedir,exist_ok=True)
    os.makedirs(impainted_imagedir,exist_ok=True)

    # 总共需要修复多少图片
    num_images = len(aligned_images)
    # 总共可以分为多少的batch来进行处理
    num_batches = int(np.ceil(num_images / batch_size))

    for idx in range(num_batches):
        # 对于每一个batch的图片进行如下处理
        lidx = idx * batch_size
        hidx = min(num_images, (idx + 1) * batch_size)
        realBatchSize = hidx - lidx

        batch_images = [get_tensor_image(imgpath) for imgpath in aligned_images[lidx:hidx]]
        batch_images = torch.stack(batch_images).to(device)
        # if realBatchSize < args.batch_size:
        #     print("number of batch images : ", realBatchSize)
        #     # 如果需要修补的图片没有一个batch那么多，用0来填充
        #     batch_images = np.pad(batch_images, [(0, args.batch_size - realBatchSize), (0, 0), (0, 0), (0, 0)], "constant")
        #     batch_images = batch_images.astype(np.float32)
        
        # 输入的原始图片已经准备好，开始准备mask
        # 暂时只提供中心mask
        mask = torch.ones(size=image_shape).to(device)
        imageCenterScale = 0.3
        lm = int(image_size * imageCenterScale)
        hm = int(image_size * (1 - imageCenterScale))
        # 将图像中心mask为0
        mask[:,lm:hm, lm:hm] = 0.0
        #print(batch_images.shape)
        #print(mask.shape)
        #return batch_images,mask
        #masked_batch_images = torch.mul(batch_images, mask).to(device)
        images[:,:,32:96,32:96]=mask
        masked_batch_images=images.to(device)

        # 先保存一下原始图片和masked图片
        save_tensor_images(batch_images.detach(),
                   os.path.join(source_imagedir,"source_image_batch_{}.png".format(idx)))
    
        save_tensor_images(masked_batch_images.detach(), os.path.join(masked_imagedir, "masked_image_batch_{}.png".format(idx)))

       
        z_hat = torch.rand(size=[realBatchSize,z_dim,1,1],dtype=torch.float32,requires_grad=True,device=device)
        z_hat.data.mul_(2.0).sub_(1.0)
        opt = optim.Adam([z_hat],lr=0.0001)       
        print("start impainting iteration for batch : {}".format(idx))
        v=torch.tensor(0,dtype=torch.float32,device=device)
        m=torch.tensor(0,dtype=torch.float32,device=device)
        
        for iteration in range(num_iters):
            # 对每一个batch的图像分别迭代impainting
            if z_hat.grad is not None:
                z_hat.grad.data.zero_()
            generator.zero_grad()
            discriminator.zero_grad()
            batch_images_g = generator(z_hat)
            batch_images_g_masked = torch.mul(batch_images_g,mask) 
            impainting_images = torch.mul(batch_images_g,(1-mask))+masked_batch_images
            if iteration % 100==0:
                # 保存impainting 图片结果
                print("\nsaving impainted images for batch: {} , iteration:{}".format(idx,iteration))
                save_tensor_images(impainting_images.detach(), os.path.join(impainted_imagedir,"impainted_image_batch_{}_iteration_{}.png".format(idx,iteration)))

            loss_context = torch.norm(
                (masked_batch_images-batch_images_g_masked),p=1)
            dis_output = discriminator(impainting_images)
#             print(dis_output)
            batch_labels = torch.full((realBatchSize,), 1, device=device)
            loss_perceptual = criteria(dis_output,batch_labels)
            
            total_loss = loss_context + lamd*loss_perceptual
            print("\r batch {} : iteration : {:4} , context_loss:{:.4f},percptual_loss:{:4f}".format(idx,iteration,loss_context,loss_perceptual),end="")
            total_loss.backward()
            opt.step()

In [27]:
images,mask=impainting()


torch.Size([10, 3, 218, 178])
torch.Size([3, 64, 64])


In [37]:
images[:,:,32:96,32:96]=mask

In [41]:
impainting()

start impainting iteration for batch : 0


RuntimeError: The size of tensor a (64) must match the size of tensor b (178) at non-singleton dimension 3

In [21]:
test=torch.from_numpy(np.array([1,2,3]))

In [22]:
test.shape

torch.Size([3])

In [24]:
test[2]=1

In [25]:
test

tensor([1, 2, 1], dtype=torch.int32)

In [ ]:
import cv2
s_img = cv2.imread("smaller_image.png")
l_img = cv2.imread("larger_image.jpg")
x_offset=y_offset=50
l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img